<a href="https://colab.research.google.com/github/HulwanulAzkaP/fraud_analisis/blob/main/Analisis_Fraud_Transaksi_Keuangan_21110009.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.
import kagglehub
zeesolver_credit_card_path = kagglehub.dataset_download('zeesolver/credit-card')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Inisiasi Pyspark

In [ ]:
# Mengimpor pustaka yang diperlukan
import pandas as pd
from pyspark.sql import SparkSession

# Membuat Spark session
spark = SparkSession.builder \
    .appName("Fraud Detection for Credit Card Transactions") \
    .getOrCreate()

# Membaca dataset ke dalam Spark DataFrame
df_spark = spark.read.csv('/kaggle/input/credit-card/creditcard_2023.csv', header=True, inferSchema=True)

# Menampilkan beberapa baris awal dari dataset
print("Data Awal di Spark:")
df_spark.show(5)

# Explorasi Data Analis

In [ ]:
import pyspark.sql.functions as F

# Memeriksa nilai hilang di dalam dataset
missing_data = df_spark.select([F.sum(F.col(col).isNull().cast("int")).alias(col) for col in df_spark.columns])
print("\nNilai Hilang dalam Dataset:")
missing_data.show()

# Menangani nilai null
# Dalam dataset ini, kita anggap semua kolom harus diisi,
# kita bisa melakukan pengisian atau penghapusan atau apa pun yang sesuai dengan analisis
df_spark = df_spark.na.fill(0)  # Misalnya, mengisi dengan 0

# Menampilkan hasil setelah pengisian
print("\nData Setelah Mengisi Nilai Hilang:")
df_spark.show(5)

In [ ]:
# Memeriksa dan mengonversi tipe data (jika perlu)
for col in df_spark.columns:
    print(f"Tipe data {col}: {df_spark.schema[col].dataType}")

# Mengonversi kolom "Amount" ke float jika belum
df_spark = df_spark.withColumn("Amount", df_spark["Amount"].cast("double"))

# Pastikan kolom Class menjadi integer (0 dan 1) untuk model klasifikasi
df_spark = df_spark.withColumn("Class", df_spark["Class"].cast("integer"))  # 0 = legitimate, 1 = fraudulent

print("\nTipe Data Kolom Setelah Konversi:")
df_spark.printSchema()

# Preprocessing Model

In [ ]:
# Membagi dataset menjadi set latih dan set uji
train_data, test_data = df_spark.randomSplit([0.7, 0.3], seed=42)
print(f"Jumlah data latih: {train_data.count()}, Jumlah data uji: {test_data.count()}")

In [ ]:
from pyspark.ml.feature import VectorAssembler

# Menyusun fitur menggunakan VectorAssembler
feature_columns = [f'V{i}' for i in range(1, 29)] + ['Amount']  # Menggunakan kolom V1-V28 dan Amount

assembler = VectorAssembler(inputCols=feature_columns, outputCol='features')
train_data = assembler.transform(train_data)
test_data = assembler.transform(test_data)

# Training Model

In [ ]:
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator

# Membuat model Random Forest
rf = RandomForestClassifier(featuresCol='features', labelCol='Class', numTrees=100)

# Melatih model
model_rf = rf.fit(train_data)

# Melakukan prediksi
predictions = model_rf.transform(test_data)

# Menilai model
evaluator = MulticlassClassificationEvaluator(labelCol='Class', predictionCol='prediction', metricName='accuracy')
accuracy = evaluator.evaluate(predictions)
print(f'Akurasi model: {accuracy:.2f}')

# Visualisasi Hasil Training

In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

# Mengonversi prediksi ke Pandas DataFrame untuk visualisasi
predictions_pd = predictions.select("Class", "prediction").toPandas()

# Menghitung confusion matrix
confusion_matrix = pd.crosstab(predictions_pd['Class'], predictions_pd['prediction'], rownames=['Actual'], colnames=['Predicted'])

plt.figure(figsize=(8, 6))
sns.heatmap(confusion_matrix, annot=True, fmt='d', cmap='Blues', xticklabels=['Legitimate', 'Fraudulent'], yticklabels=['Legitimate', 'Fraudulent'])
plt.xlabel('Prediksi')
plt.ylabel('Sebenarnya')
plt.title('Confusion Matrix')
plt.show()

In [ ]:
sns.countplot(x='prediction', data=predictions_pd)
plt.title('Distribusi Prediksi Model')
plt.xlabel('Prediksi (0: Legitimate, 1: Fraudulent)')
plt.ylabel('Jumlah')
plt.show()

# Kesimpulan Pelatihan Model Deteksi Penipuan
Akurasi Model: Model yang dilatih mencapai akurasi sebesar 95%. Ini menunjukkan bahwa model berhasil mengklasifikasikan transaksi menjadi 'legitimate' dan 'fraudulent' dengan sangat baik, artinya sebagian besar prediksi model adalah benar.

# Confusion Matrix:

True Positives (TP): 77910 transaksi fraudulent teridentifikasi dengan benar.
True Negatives (TN): 84853 transaksi legitimate teridentifikasi dengan benar.
False Positives (FP): 401 transaksi legitimate yang salah terklasifikasi sebagai fraudulent.
False Negatives (FN): 7423 transaksi fraudulent yang salah terklasifikasi sebagai legitimate.
Dari confusion matrix, terlihat bahwa meskipun model memiliki kesalahan dalam mengklasifikasi beberapa transaksi, proporsi True Positives dan True Negatives yang tinggi menunjukkan validitas model.

# Distribusi Prediksi:

Perbandingan prediksi menunjukkan bahwa jumlah transaksi legitimate jauh lebih tinggi dibandingkan fraudulent, yang merupakan karakteristik umum dalam dataset transaksi kartu kredit.
Model dapat dengan efektif mendeteksi fraudulent transactions meskipun proporsi mereka lebih kecil.
Potensi Peningkatan:

Meskipun model menunjukkan akurasi tinggi, ada ruang untuk perbaikan lebih lanjut. Khususnya, mengurangi jumlah False Negatives akan sangat penting, karena ini berarti transaksi fraudulent tidak terdeteksi dan berpotensi mengakibatkan kerugian bagi lembaga keuangan.
Metode lain seperti SMOTE (Synthetic Minority Over-sampling Technique) untuk menangani ketidakseimbangan kelas atau teknik ensemble lainnya bisa dipertimbangkan untuk meningkatkan performa.
Rekomendasi Tindak Lanjut:

Melakukan validasi silang untuk lebih memastikan daya tahan model pada data yang berbeda.
Menggunakan metrik lain seperti precision, recall, dan F1-score untuk memberikan pemahaman lebih dalam tentang performa model dalam konteks deteksi fraud.
Implementasi model dalam produksi dan monitor performanya secara berkala, menyesuaikan apabila ada perubahan dalam pola transaksi.